# Physical attacks - Detection

## I - Modules

In [75]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score,recall_score,ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb



## II - Charger donnée

Utilisons donnée préparée :

In [76]:
df_normal = pd.read_csv("preparation/physical_normal.csv")
df_attack_1 = pd.read_csv("preparation/physical_1.csv")
df_attack_2 = pd.read_csv("preparation/physical_2.csv")
df_attack_3 = pd.read_csv("preparation/physical_3.csv")
df_attack_4 = pd.read_csv("preparation/physical_4.csv")

In [77]:
concat_df_paths = ["preparation/physical/all_data_concat/all_data_concat_full_size.csv",
                   "preparation/physical/all_data_concat/all_data_concat_divided_by_10.csv",
                   "preparation/physical/all_data_concat/all_data_concat_divided_by_100.csv",               
                   ]
concat_df_name = ["full_size","divided_by_10", "divided_by_100"]

df_concat_div_10 = pd.read_csv("preparation/physical/all_data_concat/all_data_concat_divided_by_10.csv")
df_concat_div_100 = pd.read_csv("preparation/physical/all_data_concat/all_data_concat_divided_by_100.csv")
df_concat_full = pd.read_csv("preparation/physical/all_data_concat/all_data_concat_full_size.csv")

df_concat_list = [df_concat_full, df_concat_div_10, df_concat_div_100]

## III - Fonctions

### 0 - Fonction d'évaluation et matrice de confusion

In [78]:
# Definitions générales
dict_label_string_to_int = {'normal': 0, 'MITM': 1, 'physical fault': 2, 'scan': 3, 'DoS': 4}
dict_label_int_to_string = {0: 'normal', 1: 'MITM', 2: 'physical fault', 3: 'scan', 4: 'DoS'}

metrics_list = ['precision', 'recall', 'TPR', 'TNR', 'accuracy', 'F1-score', 'balanced_accuracy', 'Matthews_Correlation_Coefficient']
labels_list = ['DoS', 'MITM', 'normal', 'physical fault', 'scan']
model_list = ['KNN', 'MLP', 'SVM', 'RF', 'CART', 'XGBoost']
data_size_list = ['full', '/10', '/100']

index_df_physical_metrics = pd.MultiIndex.from_product([model_list, data_size_list, labels_list, metrics_list], names=['Model', 'Data Size', 'Attack', 'Metric'])

# C'est dans cet objet que l'on va ranger toutes les métriques de physical
df_physical_metrics = pd.DataFrame(columns=['Value'], index=index_df_physical_metrics)


In [79]:
def plot_confusion_matrix(y_preds, y_true, labels, conf_mat_name, model_name, save_cof_mat=False,):
    cm = confusion_matrix(y_true, y_preds,normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    
    if model_name == "Random_forest":
        int_labels = labels if isinstance(labels, list) else labels.tolist()

        # int_labels = labels.tolist()
        labels_for_fig = []

        for int_label in int_labels:
            labels_for_fig.append(dict_label_int_to_string[int_label])
        
    else:
        labels_for_fig = [(l[0:4] if isinstance(l, str) else str(l)) for l in labels]


    disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                                  display_labels=labels_for_fig)

    disp.plot(cmap="Blues", values_format="4f", ax=ax, colorbar=False)
    
    
    try:
        folder_path= f"./images/physical/Results/{model_name}/Confusion_matrix/"
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        if save_cof_mat:
            plt.savefig(f"./images/physical/Results/{model_name}/Confusion_matrix/{conf_mat_name}.png")
    except:
        print("ERREUR MATRICE DE CONFUSION")
        print(pd.unique(y_preds))
        print(pd.unique(y_true))
        print(labels)
        
    plt.title(conf_mat_name)
    plt.show()



def eval_model(y_pred, y_test, labels, conf_mat_name, model_name, save_cof_mat=False, silent=False):
    if model_name == "Random_forest":
        int_labels = labels.tolist()
        labels_for_fig = []

        for int_label in int_labels:
            labels_for_fig.append(dict_label_int_to_string[int_label])

    else:
        # Check if labels is already a list, if not, convert it
        labels_for_fig = labels if isinstance(labels, list) else labels.tolist()

    

    if not silent:
        plot_confusion_matrix(list(y_pred), list(y_test), labels_for_fig, conf_mat_name, model_name, save_cof_mat,)

    if not silent:
        print(classification_report(y_test, y_pred))
    
    # Print the confusion matrix directly
    cm = confusion_matrix(y_test, y_pred, labels=labels)


    index_df_model_size = pd.MultiIndex.from_product([labels_list, metrics_list], names=['Attack', 'Metric'])
    df_model_size = pd.DataFrame(columns=['Value'], index=index_df_model_size).fillna(0)
    
    # Compute and print metrics for each class
    for i, label in enumerate(labels_for_fig):
        tp = cm[i, i]
        fp = cm[:, i].sum() - tp
        fn = cm[i, :].sum() - tp
        tn = cm.sum() - (tp + fp + fn)

        tnr = tn / (tn + fp)  # True negative rate
        tpr = tp / (tp + fn)  # True positive rate
        fnr = fn / (fn + tp)  # False negative rate
        fpr = fp / (fp + tn)  # False positive rate


        precision = tp / (tp + fp)
        recall = tp / (tp + fn)

        mcc = (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
        

        # Pour chaque métrique, assignez la valeur correspondante dans le DataFrame
        df_model_size.at[(label, 'precision'), 'Value'] = precision
        df_model_size.at[(label, 'recall'), 'Value'] = recall
        df_model_size.at[(label, 'TPR'), 'Value'] = tpr
        df_model_size.at[(label, 'TNR'), 'Value'] = tnr
        df_model_size.at[(label, 'accuracy'), 'Value'] = (tp + tn) / (tp + tn + fp + fn)
        df_model_size.at[(label, 'F1-score'), 'Value'] = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        df_model_size.at[(label, 'balanced_accuracy'), 'Value'] = (tpr + tnr) / 2
        df_model_size.at[(label, 'Matthews_Correlation_Coefficient'), 'Value'] = mcc

        if not silent:
            print(label)
            print("tp", tp)
            print("fp", fp)
            print("fn", fn)
            print("tn", tn)
            print("\n")
            
            print(f"\nMetrics for class {label}:")
            print(f"True negative rate: {tnr:.4f}")
            print(f"True positive rate: {tpr:.4f}")
            print(f"False negative rate: {fnr:.4f}")
            print(f"False positive rate: {fpr:.4f}")

    
    return df_model_size
    
    
    

### 2 - KNN

In [80]:
def KNN_attack_detection(df, name_df="xxx", silent=False):
    new_df = df.copy()
    print("taille",len(df))
    X = new_df.drop('Label', axis=1)  # Caractéristiques
    y = new_df['Label']  # Cible
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    # k autour de 10 (à tester)
    k = 10

    
    knn_model = KNeighborsClassifier(n_neighbors=k)
    knn_model.fit(X_train, y_train)


    y_pred = knn_model.predict(X_test)
    df_model_size = eval_model(y_pred, y_test, knn_model.classes_, name_df, "KNN", True, silent=silent)

    return df_model_size

### 3 - Random Forest

In [81]:
def randomForest_attack_detector(df, name_df="xxx", silent=False):
    new_df = df.copy()
    
    new_df["Label"] = new_df["Label"].map(dict_label_string_to_int)
    
    X = new_df.drop('Label', axis=1)  # Features
    y = new_df['Label']  # Target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

    rf = RandomForestClassifier(n_estimators=20, random_state=42)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)
    df_model_size = eval_model(y_pred, y_test, rf.classes_, name_df, "Random_forest", True, silent=silent)

    return df_model_size
    

### 4 - MLP

In [82]:
def MLP_attack_detector(df, name_df="xxx", silent=False):
    new_df = df.copy()
    print("taille",len(df))
    X = new_df.drop('Label', axis=1)  # Caractéristiques
    y = new_df['Label']  # Cible
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


    clf = MLPClassifier(random_state=1, max_iter=300)
    clf.fit(X_train, y_train)


    y_pred = clf.predict(X_test)
    df_model_size = eval_model(y_pred, y_test, clf.classes_, name_df,  "MLP", True, silent=silent)

    return df_model_size
    

### 5 - SVM

In [83]:
def SVM_attack_detector(df, name_df="xxx", silent=False):
    new_df = df.copy()
    print("taille",len(df))
    X = new_df.drop('Label', axis=1)  # Caractéristiques
    y = new_df['Label']  # Cible
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


    clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    clf.fit(X_train, y_train)


    y_pred = clf.predict(X_test)
    df_model_size = eval_model(y_pred, y_test, clf.classes_, name_df, "SVM", True, silent=silent)

    return df_model_size

### 6) CART

In [84]:
def CART_attack_detector(df, name_df="xxx", model_name="xxx", silent=False):
    new_df = df.copy()
    print("taille",len(df))
    X = new_df.drop('Label', axis=1)  # Caractéristiques
    y = new_df['Label']  # Cible
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


    clf = DecisionTreeClassifier(random_state=1)
    clf.fit(X_train, y_train)


    y_pred = clf.predict(X_test)
    df_model_size = eval_model(y_pred, y_test, clf.classes_, True, "CART", model_name, silent=silent)

    return df_model_size

### 7) XGBoost

In [85]:
def XGB_attack_detector(df, name_df="xxx", model_name="xxx", silent=False):
    new_df = df.copy()
    print("taille",len(df))
    

    n=0
    labels = pd.unique(new_df["Label"])
    mapping = {}
    for label in labels:
        mapping[label] = n
        n += 1

    new_df["Label"] = new_df["Label"].map(mapping)

    
    X = new_df.drop('Label', axis=1)  # Caractéristiques
    y = new_df['Label']  # Cible
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


    xgb_model = xgb.XGBClassifier(objective="reg:linear", random_state=42)

    xgb_model.fit(X_train, y_train)


    y_pred = xgb_model.predict(X_test)
    df_model_size = eval_model(y_pred, y_test, list(range(n)), True, "XGBoost", model_name, silent=silent)

    return df_model_size

## IV - Detection

In [86]:
# Création des métriques pour l'ensemble des modèles, et pour l'ensembles des tailles de données
index = pd.MultiIndex.from_product([data_size_list, labels_list, metrics_list], names=['Data Size', 'Attack', 'Metric'])
df_KNN_metrics = pd.DataFrame(columns=['Value'], index=index)
df_Random_forest_metrics = pd.DataFrame(columns=['Value'], index=index)
df_MLP_metrics = pd.DataFrame(columns=['Value'], index=index)
df_SVM_metrics = pd.DataFrame(columns=['Value'], index=index)
df_CART_metrics = pd.DataFrame(columns=['Value'], index=index)
df_XGBoost_metrics = pd.DataFrame(columns=['Value'], index=index)

i = 0
for data_size, df in zip(data_size_list, df_concat_list):

    df_model_size_KNN = KNN_attack_detection(df, concat_df_name[i], silent=True)
    df_model_size_Random_forest = randomForest_attack_detector(df, concat_df_name[i], silent=True)
    df_model_size_MLP = MLP_attack_detector(df, concat_df_name[i], silent=True)
    df_model_size_SVM = SVM_attack_detector(df, concat_df_name[i], silent=True)
    df_model_size_CART = CART_attack_detector(df, concat_df_name[i], silent=True)
    df_model_size_XGBoost = XGB_attack_detector(df, concat_df_name[i], silent=True)
    
    i+=1

    # Iterate through each label and metric in df_model_size
    for label in labels_list:
        for metric in metrics_list:
            # Get the value from df_model_size and assign it to df_KNN_metrics
            value = df_model_size_KNN.at[(label, metric), 'Value']
            df_KNN_metrics.at[(data_size, label, metric), 'Value'] = value

            value = df_model_size_Random_forest.at[(label, metric), 'Value']
            df_Random_forest_metrics.at[(data_size, label, metric), 'Value'] = value

            value = df_model_size_MLP.at[(label, metric), 'Value']
            df_MLP_metrics.at[(data_size, label, metric), 'Value'] = value

            value = df_model_size_SVM.at[(label, metric), 'Value']
            df_SVM_metrics.at[(data_size, label, metric), 'Value'] = value

            value = df_model_size_CART.at[(label, metric), 'Value']
            df_CART_metrics.at[(data_size, label, metric), 'Value'] = value

            value = df_model_size_XGBoost.at[(label, metric), 'Value']
            df_XGBoost_metrics.at[(data_size, label, metric), 'Value'] = value
    

# Iterate through the MultiIndex of df_physical_metrics
for index in df_physical_metrics.index:
    model, data_size, label, metric = index

    # Check if the current row corresponds to the 'KNN' model
    if model == 'KNN':
        value = df_KNN_metrics.at[(data_size, label, metric), 'Value']
        df_physical_metrics.at[(model, data_size, label, metric), 'Value'] = value

    if model == '"Random_forest':
        value = df_model_size_Random_forest.at[(data_size, label, metric), 'Value']
        df_physical_metrics.at[(model, data_size, label, metric), 'Value'] = value
    
    if model == 'MLP':
        value = df_MLP_metrics.at[(data_size, label, metric), 'Value']
        df_physical_metrics.at[(model, data_size, label, metric), 'Value'] = value

    if model == 'SVM':
        value = df_SVM_metrics.at[(data_size, label, metric), 'Value']
        df_physical_metrics.at[(model, data_size, label, metric), 'Value'] = value

    if model == 'CART':
        value = df_CART_metrics.at[(data_size, label, metric), 'Value']
        df_physical_metrics.at[(model, data_size, label, metric), 'Value'] = value

    if model == 'XGBoost':
        value = df_XGBoost_metrics.at[(data_size, label, metric), 'Value']
        df_physical_metrics.at[(model, data_size, label, metric), 'Value'] = value

taille 10923


C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.8765432098765432' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:81: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:84: RuntimeWarning: invalid value encountered in scalar divide
  mcc = (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9828918322295805' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:81: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:84: RuntimeWarning: invalid value encountered in scalar divide
  mcc = (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


taille 10923


C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.8292682926829268' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision


...

C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:81: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:84: RuntimeWarning: invalid value encountered in scalar divide
  mcc = (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:89: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'recall'

taille 1092
taille 1092
taille 1092


C:\Users\mdrou\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\mdrou\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Users\mdrou\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Users\mdrou\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) ins

taille 109
taille 109
taille 109
taille 109
taille 109


C:\Users\mdrou\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\Users\mdrou\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\Users\mdrou\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\Users\mdrou\AppData\Roaming\Python\Python311\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) ins

In [87]:
# Save df_physical_metrics to a pickle file
df_physical_metrics.to_pickle("preparation/physical/df_physical_metrics.pkl")

### Test des models un par un 

In [88]:
dataframes = [df_attack_1, df_attack_2, df_attack_3, df_attack_4]
dataframes_name = ['df_attack_1', 'df_attack_2', 'df_attack_3', 'df_attack_4']

i=0
for df in dataframes:
    print("\n\n\033[1;33m" + dataframes_name[i] + "\033[0m")
    KNN_attack_detection(df, dataframes_name[i] + "_conf_mat", "KNN")
    i+=1

C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:89: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9908256880733946' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'recall'), 'Value'] = recall




df_attack_1
taille 2420


df_attack_2
taille 2104


df_attack_3
taille 1254


df_attack_4
taille 1717


C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:89: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'recall'), 'Value'] = recall
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:81: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:84: RuntimeWarning: invalid value encountered in scalar divide
  mcc = (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9230769230769231' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 

In [89]:
KNN_attack_detection(pd.concat([df_attack_1,df_attack_2,df_attack_3,df_attack_4]), "all_attacks_conf_mat", "KNN")

for paths in concat_df_paths:
    print("\n\n\033[1;33m" + paths + "\033[0m")
    df = pd.read_csv(paths)

    conf_mat_name = "physical " + os.path.basename(paths)
    

    KNN_attack_detection(df, conf_mat_name, "KNN")

taille 7495


C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.7903225806451613' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:81: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:84: RuntimeWarning: invalid value encountered in scalar divide
  mcc = (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))




preparation/physical/all_data_concat/all_data_concat_full_size.csv
taille 10923


preparation/physical/all_data_concat/all_data_concat_divided_by_10.csv
taille 1092


preparation/physical/all_data_concat/all_data_concat_divided_by_100.csv
taille 109


C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.8765432098765432' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:81: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:84: RuntimeWarning: invalid value encountered in scalar divide
  mcc = (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.6666666666666666' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_m

### 3 - Random Forest

In [91]:
# dataframes = [df_attack_1, df_attack_2, df_attack_3, df_attack_4]
# dataframes_name = ['df_attack_1', 'df_attack_2', 'df_attack_3', 'df_attack_4']

# i=0
# for df in dataframes:
#     print("\n\n\033[1;33m" + dataframes_name[i] + "\033[0m")
#     randomForest_attack_detector(df, dataframes_name[i] + "_conf_mat")
#     i+=1

In [92]:
randomForest_attack_detector(pd.concat([df_attack_1,df_attack_2,df_attack_3,df_attack_4]), "all_attack_conf_mat", "Random_forest")

C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9830205540661304' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision


Value
Attack         Metric                                    
DoS            precision                         0.960000
               recall                            0.923077
               TPR                               0.923077
               TNR                               0.998618
               accuracy                          0.995997
               F1-score                          0.941176
               balanced_accuracy                 0.960847
               Matthews_Correlation_Coefficient  0.939299
MITM           precision                         0.917526
               recall                            0.946809
               TPR                               0.946809
               TNR                               0.987796
               accuracy                          0.982655
               F1-score                          0.931937
               balanced_accuracy                 0.967302
               Matthews_Correlation_Coefficient  0.922151
normal         precision                         0.983021
               recall                            0.983021
               TPR                               0.983021
               TNR                               0.950000
               accuracy                          0.974650
               F1-score                          0.983021
               balanced_accuracy                 0.966510
               Matthews_Correlation_Coefficient  0.933021
physical fault precision                         1.000000
               recall                            0.978102
               TPR                               0.978102
               TNR                               1.000000
               accuracy                          0.997999
               F1-score                          0.988930
               balanced_accuracy                 0.989051
               Matthews_Correlation_Coefficient  0.987903
scan           precision                         0.000000
               recall                            0.000000
               TPR                               0.000000
               TNR                               0.998663
               accuracy                          0.996664
               F1-score                          0.000000
               balanced_accuracy                 0.499332
               Matthews_Correlation_Coefficient -0.001637

In [93]:
randomForest_attack_detector(pd.concat([df_normal,df_attack_1,df_attack_2,df_attack_3,df_attack_4]), "all_attack_&_normal_conf_mat", "Random_forest")

C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9828918322295805' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:81: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:84: RuntimeWarning: invalid value encountered in scalar divide
  mcc = (tp*tn - fp*fn) / np.sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))


Value
Attack         Metric                                    
DoS            precision                         0.972222
               recall                            0.958904
               TPR                               0.958904
               TNR                               0.999053
               accuracy                          0.997712
               F1-score                          0.965517
               balanced_accuracy                 0.978979
               Matthews_Correlation_Coefficient  0.964358
MITM           precision                         0.966292
               recall                            0.877551
               TPR                               0.877551
               TNR                               0.996983
               accuracy                          0.986270
               F1-score                          0.919786
               balanced_accuracy                 0.937267
               Matthews_Correlation_Coefficient  0.913563
normal         precision                         0.982892
               recall                            0.994972
               TPR                               0.994972
               TNR                               0.921519
               accuracy                          0.981693
               F1-score                          0.988895
               balanced_accuracy                 0.958246
               Matthews_Correlation_Coefficient  0.937389
physical fault precision                         0.983740
               recall                            0.975806
               TPR                               0.975806
               TNR                               0.999030
               accuracy                          0.997712
               F1-score                          0.979757
               balanced_accuracy                 0.987418
               Matthews_Correlation_Coefficient  0.978553
scan           precision                              NaN
               recall                            0.000000
               TPR                               0.000000
               TNR                               1.000000
               accuracy                          0.999085
               F1-score                          0.000000
               balanced_accuracy                 0.500000
               Matthews_Correlation_Coefficient       NaN

### 3 - MLP

In [94]:
dataframes = [df_attack_1, df_attack_2, df_attack_3, df_attack_4]
dataframes_name = ['df_attack_1', 'df_attack_2', 'df_attack_3', 'df_attack_4']

i=0
for df in dataframes:
    print("\n\n\033[1;33m" + dataframes_name[i] + "\033[0m")
    MLP_attack_detector(df, dataframes_name[i] + "_conf_mat", "MLP")
    i+=1



df_attack_1
taille 2420


C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.9203539823008849' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.4444444444444444' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision




df_attack_2
taille 2104


df_attack_3
taille 1254


df_attack_4
taille 1717


C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.6304347826086957' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:81: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:84: RuntimeWarning: invalid value encountered in scalar divide
  mcc = (tp*tn - fp*fn) / 

### 4 - SVM

In [95]:
dataframes = [df_attack_1, df_attack_2, df_attack_3, df_attack_4]
dataframes_name = ['df_attack_1', 'df_attack_2', 'df_attack_3', 'df_attack_4']

i=0
for df in dataframes:
    print("\n\n\033[1;33m" + dataframes_name[i] + "\033[0m")
    SVM_attack_detector(df, dataframes_name[i] + "_conf_mat", "SVM")
    i+=1



df_attack_1
taille 2420


df_attack_2
taille 2104


df_attack_3
taille 1254


df_attack_4
taille 1717


C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.8962264150943396' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'precision'), 'Value'] = precision
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:89: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_model_size.at[(label, 'recall'), 'Value'] = recall
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:81: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp + fp)
C:\Users\mdrou\AppData\Local\Temp\ipykernel_1944\1663061583.py:84: RuntimeWarning: invalid value encountered in scalar divide
  mcc = (tp*tn - fp*fn) / np.sqr